In [ ]:
# default_exp utils

# 进阶的爬虫
> 1. 更复杂的请求
    * 无headers
    * 有headers
    * 有cookies
* JSON 存储数据
* 自动识别列表
* 自动识别下一页
* 自动识别列表上的内容

* [Requests文档](https://requests.readthedocs.io/en/master/)
* [BeautifulSoup文档](https://beautiful-soup-4.readthedocs.io/en/latest/index.html)


In [ ]:
# export
import requests,json,re
from bs4 import BeautifulSoup,Tag
from collections import Counter

In [ ]:
urls = '''
https://www.zhihu.com/topic/19554834/hot
https://book.douban.com/tag/%E5%A4%96%E5%9B%BD%E6%96%87%E5%AD%A6
https://search.bilibili.com/all?keyword=%E5%A4%A7%E7%88%B7&from_source=banner_search
https://github.com/trending
https://search.jd.com/Search?keyword=%E7%94%B5%E8%84%91&enc=utf-8&wq=%E7%94%B5%E8%84%91&pvid=254ed16260384a18a34c1c452b6db180
https://bj.lianjia.com/ershoufang/
http://www.rrys2019.com/html/top/week_fav_list.html
https://weixin.sogou.com/weixin?type=2&ie=utf8&s_from=hotnews&query=%E9%9B%B7%E7%A5%9E%E5%B1%B1%E6%9C%BA%E5%99%A8%E4%BA%BA%E4%B8%8A%E5%B2%97
http://ciligou.top/search?word=%E6%B5%81%E6%B5%AA%E5%9C%B0%E7%90%83
'''
url_list = urls.strip().split('\n')
soup_list = []

for url in url_list:
    print('start:',url)
    res = requests.get(url,headers={'user-agent':'Mozilla/5.0'})
    if res.status_code == 200:
        soup = BeautifulSoup(res.text,'lxml')
        soup_list.append(soup)
    else:
        print(res,res.text)

start: https://www.zhihu.com/topic/19554834/hot
start: https://book.douban.com/tag/%E5%A4%96%E5%9B%BD%E6%96%87%E5%AD%A6
start: https://search.bilibili.com/all?keyword=%E5%A4%A7%E7%88%B7&from_source=banner_search
start: https://github.com/trending
start: https://search.jd.com/Search?keyword=%E7%94%B5%E8%84%91&enc=utf-8&wq=%E7%94%B5%E8%84%91&pvid=254ed16260384a18a34c1c452b6db180
start: https://bj.lianjia.com/ershoufang/
start: http://www.rrys2019.com/html/top/week_fav_list.html
start: https://music.163.com/discover/playlist
start: https://weixin.sogou.com/weixin?type=2&ie=utf8&s_from=hotnews&query=%E9%9B%B7%E7%A5%9E%E5%B1%B1%E6%9C%BA%E5%99%A8%E4%BA%BA%E4%B8%8A%E5%B2%97
start: http://ciligou.top/search?word=%E6%B5%81%E6%B5%AA%E5%9C%B0%E7%90%83


In [ ]:
len(soup_list) == len(url_list)

True

## 更复杂的请求
> 以豆瓣网为例

一开始没有header的请求会被拒绝，因为服务器会把这次的请求当成是机器人，status_code是418

In [ ]:
# hide
search_query = '1234'
url = f'https://www.douban.com/search?q={search_query}'
res = requests.get(url)
res,res.text

(<Response [418]>, '')

加了header之后，服务器就把它当成了Mozilla浏览器，就有response的数据了

In [ ]:
# hide
headers = {'user-agent':'Mozilla/5.0'}
res = requests.get(url,headers=headers)
res

<Response [200]>

想要发布一个动态就需要登录状态，而cookie就是登录状态的载体，cookie就是用帐号密码登录后获取的凭证

In [ ]:
# hide
url = 'https://www.douban.com/'
data = {'comment':'hello world','ck':'zV8Z','privacy_and_reply_limit':'P,'}

In [ ]:
# export
def format_cookie_str(cookie_str):
    '把chrome拷贝过来的cookie字符转化成dict'
    cookies = {}
    for item in cookie_str.split('; '):
        k,v = item.split('=',1)
        cookies[k] = v
    return cookies

In [ ]:
cookie_str = 'bid=1i8YWHFPDwI; gr_user_id=5b798ccf-0dc3-41f7-9358-ab221ae5c248; __utmc=30149280; __utmz=30149280.1582040380.5.5.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); ll="118124"; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1582185091%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_ses.100001.8cb4=*; __utma=30149280.682236232.1580713449.1582121697.1582185095.7; ap_v=0,6.0; viewed="19672873_30243169_4233221"; gr_cs1_6bb1b2b8-0a3e-4e02-9e3c-4f359d514576=user_id%3A0; __utmt_douban=1; dbcl2="140014301:Td6zJ+yn5sA"; ck=zV8Z; gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03=7a596b44-fe53-45f0-90fa-2c24b2faa365; gr_cs1_7a596b44-fe53-45f0-90fa-2c24b2faa365=user_id%3A1; gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03_7a596b44-fe53-45f0-90fa-2c24b2faa365=true; push_noty_num=0; push_doumail_num=0; __utmv=30149280.14001; __utmt=1; ps=y; _pk_id.100001.8cb4=7bc8021c269d7e50.1580713448.6.1582185748.1582121848.; __utmb=30149280.18.10.1582185095'
cookies = format_cookie_str(cookie_str)
cookies

{'bid': '1i8YWHFPDwI',
 'gr_user_id': '5b798ccf-0dc3-41f7-9358-ab221ae5c248',
 '__utmc': '30149280',
 '__utmz': '30149280.1582040380.5.5.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
 'll': '"118124"',
 '_pk_ref.100001.8cb4': '%5B%22%22%2C%22%22%2C1582185091%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D',
 '_pk_ses.100001.8cb4': '*',
 '__utma': '30149280.682236232.1580713449.1582121697.1582185095.7',
 'ap_v': '0,6.0',
 'viewed': '"19672873_30243169_4233221"',
 'gr_cs1_6bb1b2b8-0a3e-4e02-9e3c-4f359d514576': 'user_id%3A0',
 '__utmt_douban': '1',
 'dbcl2': '"140014301:Td6zJ+yn5sA"',
 'ck': 'zV8Z',
 'gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03': '7a596b44-fe53-45f0-90fa-2c24b2faa365',
 'gr_cs1_7a596b44-fe53-45f0-90fa-2c24b2faa365': 'user_id%3A1',
 'gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03_7a596b44-fe53-45f0-90fa-2c24b2faa365': 'true',
 'push_noty_num': '0',
 'push_doumail_num': '0',
 '__utmv': '30149280.14001',
 '__utmt': '1',
 'ps': 'y',
 '_pk_id.100001.8cb

In [ ]:
# hide
res = requests.post(url,headers=headers,data=data,cookies=cookies)
res.text

'<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>豆瓣</title>\n    \n    \n<meta content="提供图书、电影、音乐唱片的推荐、评论和价格比较，以及城市独特的文化生活。" name="description"/>\n\n    <link href="https://img3.doubanio.com/f/shire/db7c8ab218cded634c1180ff7b56d3ca44393bc2/css/frontpage/_init_.css" rel="stylesheet" type="text/css">\n    <script>_head_start = new Date();</script>\n    <script src="https://img3.doubanio.com/f/shire/ebac5abada5de811d65dac57b2e62a89c7ddf55a/js/core/_init_.js" data-cfg-corelib="https://img3.doubanio.com/f/shire/72ced6df41d4d158420cebdd254f9562942464e3/js/jquery.min.js"></script>\n    \n    <style type="text/css"></style>\n    <link rel="stylesheet" href="https://img3.doubanio.com/misc/mixed_static/2008baff28

In [ ]:
# hide
# 查看最近发布的5条动态
soup = BeautifulSoup(res.text)
items = soup.find_all(class_='new-status')
for item in items[:5]:
    print(item.find('p').text)

hello world
hello world
hello world
hello world
hello world


In [ ]:
# hide
type(items[0]),items[0].attrs

(bs4.element.Tag,
 {'class': ['new-status', 'status-wrapper', 'saying'],
  'data-sid': '2849308445',
  'data-uid': '140014301'})

顺便看看tag的其他属性，上下左右查到其他的tag
* item.parent
* item.parents

* item.contents
* item.children

* item.next_sibling
* item.previous_sibling

## JSON 存储
> 这个格式本质上就是个dict，在MongoDB和Redis中也是这样存储，所以这里就开始学习下

比如这次要把谁在什么时间，说了什么话记下来

In [ ]:
# hide
print(items[5]['data-sid'])
print(items[0].find(class_='text').a.text)
print(items[0].find(class_='created_at')['title'])
print(items[4].find(class_='bd').contents[1].text.replace('\n',''))

2849283969
Hawk
2020-03-04 19:37:35
hello world


In [ ]:
# hide
contents = {}
for item in items:
    _id = item['data-sid']
    name = item.find(class_='text').a.text    
    created_at = item.find(class_='created_at')['title']
    content = item.find(class_='bd').contents[1].text.replace('\n','')  
    contents[_id] = {}
    contents[_id]['name'] = name
    contents[_id]['created_at'] = created_at  
    contents[_id]['content'] = content
contents

{'2849308445': {'name': 'Hawk',
  'created_at': '2020-03-04 19:37:35',
  'content': 'hello world'},
 '2849286231': {'name': 'Hawk',
  'created_at': '2020-03-04 19:23:46',
  'content': 'hello world'},
 '2849285799': {'name': 'Hawk',
  'created_at': '2020-03-04 19:23:26',
  'content': 'hello world'},
 '2849285223': {'name': 'Hawk',
  'created_at': '2020-03-04 19:23:05',
  'content': 'hello world'},
 '2849284578': {'name': 'Hawk',
  'created_at': '2020-03-04 19:22:42',
  'content': 'hello world'},
 '2849283969': {'name': 'Hawk',
  'created_at': '2020-03-04 19:22:19',
  'content': 'hello world'},
 '2849283502': {'name': 'Hawk',
  'created_at': '2020-03-04 19:21:59',
  'content': 'hello world'},
 '2849282899': {'name': 'Hawk',
  'created_at': '2020-03-04 19:21:34',
  'content': 'hello world'},
 '2849282248': {'name': 'Hawk',
  'created_at': '2020-03-04 19:21:10',
  'content': 'hello world'},
 '2849281687': {'name': 'Hawk',
  'created_at': '2020-03-04 19:20:51',
  'content': 'hello world'},


### 保存和读取json

In [ ]:
# hide
with open('./data/01_douban.json', 'w') as f:
    json.dump(contents,f)

In [ ]:
# hide
with open('./data/01_douban.json', 'r') as f:
    contents = json.loads(f.read())
contents['2726525573']

{'name': 'Malorie',
 'created_at': '2019-12-14 10:18:01',
 'content': "21 身份危机 |Model MinoritySomeone like me can be a real nightmare, completely aware But I'd rather be a real nightmare than die unaware So save me your prayers 一直很想写这篇日记，却因为太贴近真实的自我..."}

## 自动识别列表
> 思路就是看谁有最多的相同类的children（目前只支持静态页面,文字列表）

常见问题：容易识别成目录，特征为多层级列表，而正文内容则不会再嵌套列表

In [ ]:
#export
def get_children(soup): return [c for c in soup.children if isinstance(c,Tag)]

def is_content_list(soup): 
    '判断是否为内容列表，条件为：子集是重复的相同的tag，重复次数大于5'
    children = get_children(soup)
    if len(children) > 1 and len(soup.text.strip()) > 0:
        '统计tag name'
        tag_counter = Counter([c.name for c in children])
        max_tag_name,max_tag_count = Counter([c.name for c in children]).most_common(1)[0]
        main_tag_pct = max_tag_count/len(children)
        # 这里还有优化空间，就是class一致程度高于多少
        if max_tag_count >= 5 and main_tag_pct > 0.9:
#             print('候选：\n',soup.name,soup.attrs,tag_counter,len(soup.text),get_class_count(soup),len(soup.text)/len([i for i in soup.stripped_strings]))
            return True
    return False

def get_class_count(soup):
    class_list = []
    for s in soup.find_all(True):
        if 'class' in s.attrs:
            class_list += s.attrs['class']
    class_list = list(set(class_list))
    return len(class_list)

def mean(l):
    return sum(l)/len(l)

def find_main_list(soup):
    '特征：文字内容多；一条数据的样式&嵌套丰富；区别于目录，都是很短的词'
    score = 0
    main_list = {'name':'unrecognized','attrs':{}}
    
    items = soup.find_all(True) 
    for item in items:
        if is_content_list(item):
            text_count = len(item.text)
            class_count = get_class_count(item)
#             text_avg_lenth = mean([len(i) for i in item.stripped_strings])
            text_max_lenth = max([len(i) for i in item.stripped_strings])
            
            new_score = text_count*class_count*text_max_lenth
            if new_score > score:
                score = new_score
                main_list = item
    first_child = get_children(main_list)[0]
    print('终选：\n',main_list.name,main_list.attrs,first_child.name,first_child.attrs,score)
    return main_list

In [ ]:
for url,soup in zip(url_list,soup_list):
    if '163.com' not in url:
#     if 'ciligou' in url:
        print(url)
        soup_test = soup.body
#         print(soup.body)
        main_list = find_main_list(soup.body)

https://www.zhihu.com/topic/19554834/hot
终选：
 div {'class': []} div {'class': ['List-item', 'TopicFeedItem'], 'tabindex': '0'} 18537168
https://book.douban.com/tag/%E5%A4%96%E5%9B%BD%E6%96%87%E5%AD%A6
终选：
 ul {'class': ['subject-list']} li {'class': ['subject-item']} 6528357
https://search.bilibili.com/all?keyword=%E5%A4%A7%E7%88%B7&from_source=banner_search
终选：
 ul {'type': 'video', 'class': ['video-list', 'clearfix']} li {'class': ['video-item', 'matrix']} 6437548
https://github.com/trending
终选：
 div {} article {'class': ['Box-row']} 60059400
https://search.jd.com/Search?keyword=%E7%94%B5%E8%84%91&enc=utf-8&wq=%E7%94%B5%E8%84%91&pvid=254ed16260384a18a34c1c452b6db180
终选：
 ul {'class': ['gl-warp', 'clearfix'], 'data-tpl': '1'} li {'data-sku': '1555771170', 'class': ['gl-item']} 111600720
https://bj.lianjia.com/ershoufang/
终选：
 ul {'class': ['sellListContent'], 'log-mod': 'list'} li {'class': ['clear', 'LOGVIEWDATA', 'LOGCLICKDATA'], 'data-lj_view_evtid': '21625', 'data-lj_evtid': '2162

In [ ]:
soup_test.prettify()

'<body>\n <!--wrap-->\n <div class="Information_outer_container" id="wrapper">\n  <div id="head">\n   <div id="Header_wrapper_other_style">\n    <div id="Header_logo">\n     <a data-reactid="5" href="/">\n      <img alt="磁力狗" src="/static/img/logomini.gif"/>\n     </a>\n    </div>\n    <div id="Header_right_nav">\n     <div id="Category_container">\n      <div class="Category_category_nav_wrapper">\n       <span class="Category_category_icon">\n        <i class="iconfont icon-jiugongge-copy">\n        </i>\n       </span>\n       <span class="Category_category_icon">\n        <i class="iconfont icon-gengduo1">\n        </i>\n       </span>\n      </div>\n     </div>\n     <!-- \t\t\t\t\t<div class="LoginBtn_wrapper">\n\t\t\t\t\t\t登录\n\t\t\t\t\t</div> -->\n    </div>\n   </div>\n  </div>\n  <div class="container" id="Search_container">\n   <div id="Header_wrapper_search_style">\n    <div id="Header_search_form">\n     <form action="/search" id="SearchForm_form">\n      <div id="SearchFo

## 自动识别下一页
> 目前只能识别静态页面

In [ ]:
#export
def is_next_page(tag):
    if tag.name == 'a':
        for text in ['下一页','后页']:
            if text in tag.text:
                return True
    else:
        return False
    
def get_next_page_url(soup):
    next_page = soup.find(is_next_page)
    if (next_page):
        return next_page['href']
    else:
        print('未识别出下一页')
        return None 

In [ ]:
url = 'https://book.douban.com/tag/%E5%A4%96%E5%9B%BD%E6%96%87%E5%AD%A6'
res_sample = requests.get(url,headers={'user-agent':'Mozilla/5.0'})
soup_sample = BeautifulSoup(res_sample.text)

get_next_page_url(soup_sample)

'/tag/外国文学?start=20&type=T'

## 自动识别列表内容
> 思路就是样式一样的，是一类的内容

目前支持的几种内容类型：
1. img link
* a link
* text

In [ ]:
#export

def get_prefix(soup):
    if 'class' in soup.attrs and len(soup.attrs['class']) > 0:
        prefix = soup.name+'.'+soup.attrs['class'][0]
    elif 'id' in soup.attrs:
        prefix = soup.name+'.'+soup.attrs['id']
    else:
        prefix = soup.name
    return prefix

def get_item_data(soup,db={}):
    children = get_children(soup)
    if len(children) > 0:
        for c in children:
            db[get_prefix(c)] = {}
            get_item_data(c,db[get_prefix(c)])
    else:
        if soup.name == 'a':
            db['href'] = soup['href']
            if 'title' in soup.attrs:
                db['title'] = soup['title']
        elif soup.name == 'img':
            db['src'] = soup['src']
        else:
            db['text'] = soup.text.strip()
    return db

def find_item_id(soup):
    # 找到内容id，思路是a标签，有title，有href，然后取里面数字    
    for a in soup.find_all('a'):
        if 'title' in a.attrs and 'href' in a.attrs:
            id = re.search(r'\d+',a['href']).group(0)
            return id


In [ ]:
items = get_children(main_list)
contents = items[0].find_all(True)

In [ ]:
items[0].prettify(formatter="minimal")

'<div class="List-item TopicFeedItem" tabindex="0">\n <div class="ContentItem ArticleItem" data-zop=\'{"authorName":"蔓玫","itemId":111342516,"title":"有关「美」与「生命力」，兰花给过我们怎样的启迪？","type":"article"}\' itemprop="article" itemscope="" itemtype="http://schema.org/SocialMediaPosting">\n  <h2 class="ContentItem-title">\n   <a data-za-detail-view-element_name="Title" href="//zhuanlan.zhihu.com/p/111342516" rel="noopener noreferrer" target="_blank">\n    有关「美」与「生命力」，兰花给过我们怎样的启迪？\n   </a>\n  </h2>\n  <div class="ContentItem-meta">\n   <div class="AuthorInfo ArticleItem-authorInfo AuthorInfo--plain" itemprop="author" itemscope="" itemtype="http://schema.org/Person">\n    <meta content="蔓玫" itemprop="name"/>\n    <meta content="https://pic4.zhimg.com/50/v2-e255e8d3ffd3b76ec46bb5522d937fcd_is.jpg" itemprop="image"/>\n    <meta content="https://www.zhihu.com/people/rosecrimson" itemprop="url"/>\n    <meta itemprop="zhihu:followerCount"/>\n    <span class="UserLink AuthorInfo-avatarWrapper">\n     <div c

In [ ]:
for c in contents:
    if c.text:
        print((c.text))

有关「美」与「生命力」，兰花给过我们怎样的启迪？蔓玫.css-1cd9gw4{margin-left:.3em;}.css-1pc1mic{box-sizing:border-box;margin:0;min-width:0;color:#175199;display:inline-block;width:18px;height:18px;margin-left:.3em;}.css-18biwo{display:-webkit-inline-box;display:-webkit-inline-flex;display:-ms-inline-flexbox;display:inline-flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;}​植物、花卉、植物学 等 4 个话题的植物等 话题优秀回答者「那时的黛西毕竟还年轻，她那虚华的世界里充溢着兰花的芬芳、社交的愉悦和乐队的欢歌，正是那些歌舞乐曲为逝水年华定下了基调，用新的旋律演绎着人世的启示和忧伤。」 ——菲茨杰拉德《了不起的盖茨比》一个不负责任的猜想：若要问问观众们，电影《了不起…阅读全文​​赞同 27​​7 条评论​分享​收藏
有关「美」与「生命力」，兰花给过我们怎样的启迪？
有关「美」与「生命力」，兰花给过我们怎样的启迪？
蔓玫.css-1cd9gw4{margin-left:.3em;}.css-1pc1mic{box-sizing:border-box;margin:0;min-width:0;color:#175199;display:inline-block;width:18px;height:18px;margin-left:.3em;}.css-18biwo{display:-webkit-inline-box;display:-webkit-inline-flex;display:-ms-inline-flexbox;display:inline-flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;}​植物、花